<a href="https://colab.research.google.com/github/ternithinator/test2/blob/main/Genomics_Workshop_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<video src="https://raw.githubusercontent.com/PoODL-CES/PoODL_NGS_workshop.io/main/Nature%20of%20Science.mp4" controls autoplay loop width="400" align="right"></video>

## Genomics Learning Workshop: NGS Analysis Pipeline

Easy-to-follow workflow for processing Illumina whole-genome resequencing reads for population genomics. Steps include trimming, mapping, sorting, variant calling, variant filtering, PCA, and ADMIXTURE analysis. For more details, see the bottom of this notebook and visit the GitHub repository.
Repository link: [Genomics Learning Workshop](https://github.com/PoODL-CES/Genomics_learning_workshop)

This workshop introduces:
- Handling raw FASTQ files
- Performing quality control and trimming
- Mapping reads to a reference genome
- Calling and filtering variants
- Running PCA and ADMIXTURE for population analysis


In [ ]:
# Miniconda installation and environment setup for Colab NGS Workshop

# Download and install Miniconda (skip if already installed)
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /usr/local/miniconda

import sys, os
sys.path.append('/usr/local/miniconda/lib/python3.8/site-packages')
os.environ['PATH'] = "/usr/local/miniconda/bin:" + os.environ['PATH']

# Explicitly clear potentially problematic environment variables from Python's os.environ
if 'CONDA_PREFIX' in os.environ:
    del os.environ['CONDA_PREFIX']
if 'CONDA_ENVS_PATH' in os.environ:
    del os.environ['CONDA_ENVS_PATH']

# Accept ToS for main and R conda channels
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Create new conda environment for your workshop if it doesn't exist
!if ! conda info --envs | grep -w 'workshop_ngs'; then \
    conda create -y -n workshop_ngs python=3.7; \
else \
    echo "Environment 'workshop_ngs' already exists, skipping creation."; \
fi

# Install necessary bioinformatics tools into the environment
!conda install -y -n workshop_ngs -c bioconda -c conda-forge trim-galore samtools fastqc bwa gatk4=4.3.0.0

# Verify tool versions inside activated environment in one shell session
!bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && \
          trim_galore --version && samtools --version && fastqc --version && \
          bwa --version && gatk --version"

To activate the `workshop_ngs` environment for specific commands and ensure its tools are used, you can wrap your commands within a `bash -c "source /usr/local/miniconda/bin/activate workshop_ngs && your_command_here"` block. This ensures the environment is properly sourced before the command runs. Let's try listing the environment contents and checking `CONDA_PREFIX` within that activated session:

In [ ]:
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && conda activate workshop_ngs && \
          echo '--- Environment activated ---' && \
          echo 'CONDA_PREFIX in activated env: '$CONDA_PREFIX && \
          echo '--- Listing contents of activated env ---' && \
          conda list"

We’ll be using real WGS resequencing data for this workshop. The dataset is a small subset for demonstration purposes. We will download the data from (https://zenodo.org/records/14258052) . Step 1 would be to make a parent directory to sort all the data in one place.


In [ ]:
%%bash

mkdir -p fastq_files
ls -F

Now, let's navigate into the `fastq_files` directory.

In [ ]:
%%bash

cd fastq_files/ && pwd

Let's organize the download process by first ensuring the `fastq_files` directory exists, then navigating into it to download all the FASTQ files directly to their intended location.

In [ ]:
# Define the list of FASTQ file links
fastq_links = [
    "https://zenodo.org/records/14258052/files/BEN_CI16_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_CI16_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW10_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_NW10_sub_2.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI18_sub_1.fq.gz",
    "https://zenodo.org/records/14258052/files/BEN_SI18_sub_2.fq.gz",
]

# Create the directory if it doesn't exist
!mkdir -p fastq_files

# Clean up any existing fastq files and duplicates before downloading
!rm -f fastq_files/*.fq.gz*

# Change to the directory and download files
for link in fastq_links:
    # Use -nc (no-clobber) to avoid re-downloading if file exists and is complete,
    # and -P . to save to current directory (fastq_files after cd)
    !bash -c "cd fastq_files && wget -nc -P . -q {link}"

# List the contents of the fastq_files directory to confirm all downloads
!ls -F fastq_files/

To visualise the content of the directory 'fastq_files' , to confirm that all the files have been successfully downloaded.

Once all the fastq files are downloaded, the next would be to run "FastQC" tool on the raw reads for the quality assessment step before the downstream analysis part.

***Why it's important:*** Poor-quality reads can lead to false variant calls or poor mapping, so quality control is crucial.

To do so, first, let's create a directory to store the FastQC reports.

In [ ]:
%%bash

mkdir -p fastqc_results
ls -F

Now, we will run FastQC on all the FASTQ files within the `workshop_ngs` environment and save the reports to the `fastqc_results` directory.

In [ ]:
%%time
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && conda activate workshop_ngs && \
          fastqc fastq_files/*.fq.gz -o fastqc_results"

# List the generated FastQC reports
!ls -F fastqc_results/

The .html files generated by FastQC are standard web pages, which means you can visualize them by opening them in any web browser!

Here's how you can access them from Colab:
 On the left-hand side of your Colab interface, there's usually a file explorer icon (a folder). Click on it to open the file browser. Navigate to the fastqc_results directory. You'll see all the .html files listed there. You can right-click on any .html file and select 'Download' to save it to your local machine, then open it with your preferred web browser.




Post qualitty assessment, the next step will be the trimming of the adapters or low-quality bases using tools such as Trim-galore. This step will ensure producing clean FASTQ files, ready for mapping.

Step 1: We'll create a directory to store the trimmed FASTQ files.

In [ ]:
!mkdir -p trimmed_fastq_files
!ls -F

Now, we will activate the `workshop_ngs` conda environment and run `Trim Galore!` on all the FASTQ files. We'll specify that they are paired-end reads and direct the output to the `trimmed_fastq_files` directory.

Trim Galore automatically detects and removes common adapter sequences and performs quality trimming. The `--paired` option is essential for correctly processing paired-end reads.

In [ ]:
%%time
# Run Trim Galore! within the activated conda environment
# We iterate through the samples to process paired-end reads correctly

import os

# Get a list of fastq files programmatically
all_files = os.listdir('fastq_files/')
fastq_files_list = [os.path.join('fastq_files', f) for f in all_files if f.endswith('.fq.gz')]

# Extract unique sample prefixes (e.g., BEN_CI16, BEN_NW10, BEN_SI18) from the filenames
sample_prefixes = sorted(list(set([f.split('/')[-1].split('_sub_')[0] for f in fastq_files_list])))

print(f"Detected FASTQ files: {fastq_files_list}")
print(f"Detected sample prefixes: {sample_prefixes}")

for prefix in sample_prefixes:
    read1 = f"fastq_files/{prefix}_sub_1.fq.gz"
    read2 = f"fastq_files/{prefix}_sub_2.fq.gz"
    output_dir = "trimmed_fastq_files"

    # Use bash -c to activate conda env and run trim_galore
    # --paired: for paired-end reads
    # -o: specify output directory
    # --fastqc is removed as per user request to avoid generating html/zip files
    !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && conda activate workshop_ngs && \
              trim_galore --paired {read1} {read2} -o {output_dir}"

    # Remove unwanted output files for the current prefix
    !rm -f {output_dir}/{prefix}_sub_*.fq.gz_trimming_report.txt
    !rm -f {output_dir}/{prefix}_sub_*_fastqc.html
    !rm -f {output_dir}/{prefix}_sub_*_fastqc.zip

# List the contents of the trimmed_fastq_files directory to confirm only _val files remain
!ls -F trimmed_fastq_files/

In [ ]:
%%time

import os

# Direct paths for this single sample
read1 = "fastq_files/BEN_CI16_sub_1.fq.gz"
read2 = "fastq_files/BEN_CI16_sub_2.fq.gz"
output_dir = "trimmed_fastq_files"

# Make output directory if missing
os.makedirs(output_dir, exist_ok=True)

print("\n--- Running Trim Galore for BEN_CI16 ---")

# Run Trim Galore inside conda environment
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
          conda activate workshop_ngs && \
          trim_galore --paired {read1} {read2} -o {output_dir}"

# Remove unwanted reports and QC files
!rm -f {output_dir}/BEN_CI16_sub_*.fq.gz_trimming_report.txt
!rm -f {output_dir}/BEN_CI16_sub_*_fastqc.html
!rm -f {output_dir}/BEN_CI16_sub_*_fastqc.zip

# Show trimmed output files
print('\n--- Trimmed files in trimmed_fastq_files/ ---')
!ls -F trimmed_fastq_files/


### Downloading reference files for mapping

To prepare for the read mapping step, we need to download the reference genome and its associated index files. These files will be stored in a new directory called `reference`.

In [ ]:
%%time
!rm -rf reference
reference_tar = "https://zenodo.org/records/17878528/files/reference.tar.gz?download=1"

# Download the tar.gz file
!wget -q $reference_tar -O reference.tar.gz

# Extract it — this will unpack into its own folder (likely 'reference/')
!tar -xzvf reference.tar.gz

# List extracted contents
!ls -F reference/

# Mapping reads to the Reference genome:
Here we:

Use an aligner like BWA or Bowtie2 to map the cleaned reads to a reference genome

Convert output from SAM to BAM format (compressed and binary)

Sort the BAM files by genomic coordinates (using samtools sort)


Index the BAM files so tools can access them efficiently

*Why it's important:* Proper alignment is the foundation for all downstream analysis. Sorting/indexing ensures quick and efficient variant detection.



Create a new directory named `mapped_reads` to store output alignment files, then index the reference genome located at `reference/GCA_021130815.1_PanTigT.MC.v3_genomic.fna` using `bwa index` for efficient read mapping. After indexing, map the trimmed paired-end reads from each sample to the indexed reference genome using `bwa mem`, converting the output to sorted BAM files and indexing them using `samtools`.

In [ ]:
!mkdir -p mapped_reads

# Paths for this sample
REFERENCE_GENOME = "reference/GCA_021130815.1_PanTigT.MC.v3_genomic.fna"

read1 = "trimmed_fastq_files/BEN_CI16_sub_1_val_1.fq.gz"
read2 = "trimmed_fastq_files/BEN_CI16_sub_2_val_2.fq.gz"
output_bam = "mapped_reads/BEN_CI16.bam"

print("\n--- Mapping BEN_CI16 ---")

# Map + convert to BAM + sort BAM
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
          conda activate workshop_ngs && \
          bwa mem -M -R '@RG\tID:BEN_CI16\tSM:BEN_CI16\tPL:ILLUMINA' \
          {REFERENCE_GENOME} {read1} {read2} | \
          samtools view -bS - | \
          samtools sort -o {output_bam} -"

# Index sorted BAM
#print("Indexing BEN_CI16...")
#!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
 #         conda activate workshop_ngs && \
  #        samtools index {output_bam}"

# List output files
!ls -lh mapped_reads/


#Mark Duplicates

This command detects PCR duplicates and optical duplicates in the aligned reads and removes them. Duplicate reads can bias downstream analyses such as variant calling, so removing them improves data quality.

Explanation of the flags we use are given below

-I : Input BAM file. This is the sorted BAM produced from the mapping step.

-O :	Output BAM file after duplicates have been removed.

-M :	File where GATK writes duplication statistics (number of duplicates, percent duplication, etc.)

--REMOVE_DUPLICATES true :	Instead of just marking duplicates, this flag instructs GATK to remove them entirely from the BAM file.



---




**After mark duplicates we also need to Index the deduplicated bam file**

Creates an index (.bai) file for the deduplicated BAM file.
This index enables efficient random access to the BAM file and is required for many downstream tools






In [ ]:
!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
          conda activate workshop_ngs && \
          gatk MarkDuplicates \
            -I BEN_CI16.bam \
            -O BEN_CI16.deduplicated.bam \
            -M BEN_CI16.metrics.txt \
            --REMOVE_DUPLICATES true"


print("\n--- Indexing deduplicated BAM ---")

!bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
          conda activate workshop_ngs && \
          samtools index mapped_reads/BEN_CI16.deduplicated.bam"

# List output files
!ls -lh mapped_reads/

#Tutorial Exercise

Once you have performed trimming and mapping and completed till indexing steps given above, same can be done for the other two fastq files which we have downloaded (BEN_NW10 and BEN_SI18).

Steps for the same are given below.

**Trimming**

In [ ]:
%%time

import os

samples = ["BEN_NW10", "BEN_SI18"]
output_dir = "trimmed_fastq_files"
os.makedirs(output_dir, exist_ok=True)

for prefix in samples:
    read1 = f"fastq_files/{prefix}_sub_1.fq.gz"
    read2 = f"fastq_files/{prefix}_sub_2.fq.gz"

    print(f"\n--- Running Trim Galore for {prefix} ---")

    # Run Trim Galore inside conda environment
    !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
              conda activate workshop_ngs && \
              trim_galore --paired {read1} {read2} -o {output_dir}"

    # Remove extra report files
    !rm -f {output_dir}/{prefix}_sub_*.fq.gz_trimming_report.txt
    !rm -f {output_dir}/{prefix}_sub_*_fastqc.html
    !rm -f {output_dir}/{prefix}_sub_*_fastqc.zip

print("\n--- Trimmed files ---")
!ls -F trimmed_fastq_files/


**Mapping**

In [ ]:
%%time

REFERENCE_GENOME = "reference/GCA_021130815.1_PanTigT.MC.v3_genomic.fna"
os.makedirs("mapped_reads", exist_ok=True)

samples = ["BEN_NW10", "BEN_SI18"]

for prefix in samples:

    read1 = f"trimmed_fastq_files/{prefix}_sub_1_val_1.fq.gz"
    read2 = f"trimmed_fastq_files/{prefix}_sub_2_val_2.fq.gz"
    output_bam = f"mapped_reads/{prefix}.bam"

    print(f"\n--- Mapping {prefix} ---")

    # Map → convert → sort
    !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
              conda activate workshop_ngs && \
              bwa mem -M -R '@RG\tID:{prefix}\tSM:{prefix}\tPL:ILLUMINA' \
              {REFERENCE_GENOME} {read1} {read2} | \
              samtools view -bS - | \
              samtools sort -o {output_bam} -"

    # Index BAM
   # print(f"Indexing BAM for {prefix}...")
  #  !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
        #      conda activate workshop_ngs && \
      #        samtools index {output_bam}"


!ls -F mapped_reads/


**Mark Duplicates + Indexing**

In [ ]:
samples = ["BEN_NW10", "BEN_SI18"]

for prefix in samples:
    print(f"\n--- Running MarkDuplicates for {prefix} ---")

    !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
              conda activate workshop_ngs && \
              gatk MarkDuplicates \
                -I mapped_reads/{prefix}.sorted.bam \
                -O mapped_reads/{prefix}.deduplicated.bam \
                -M mapped_reads/{prefix}.metrics.txt \
                --REMOVE_DUPLICATES true"

    print(f"\n--- Indexing deduplicated BAM for {prefix} ---")

    !bash -c "source /usr/local/miniconda/etc/profile.d/conda.sh && \
              conda activate workshop_ngs && \
              samtools index mapped_reads/{prefix}.deduplicated.bam"

print("\n--- Finished MarkDuplicates + Indexing for all samples ---")
!ls -lh mapped_reads/
